## Import all required packages

In [1]:
import docplex
from docplex.mp.model import Model
import pandas as pd
import numpy as np
import math

## create model

In [2]:
model = Model(name="Jetty Scheduling inst3", log_output=True)

## input data and parameters

In [3]:
# read data from excel
data = pd.read_excel("inst3.xlsx")
U_cp = data.iloc[1:6,1:5] # time for unloading u_cp
ship_catagories = data.iloc[1:21,6:8] # ship catagories
ship_catagories.columns = ["ship","catagories"]
ship_arrival = data.iloc[1:21,9:11] # ship's arrival time
ship_arrival.columns = ["ship","arrival_time"]
# merge the two dataframe ship_catagories and ship_arrival
ship_data = pd.concat([ship_catagories, ship_arrival.iloc[:,1]], axis=1)
# sort ships by their arrival time
ship_data = ship_data.sort_values(by=["arrival_time"])

ship = ship_arrival.iloc[:,0].tolist() # list of ships (sorted in arrival time)
S = list(range(1,len(ship)+1)) # represents the arrival order of each ship s
a = ship_data.iloc[:,2].tolist() # arrival time of each ship in sequence a_s
P = list(range(1,5)) # list of stations

# define a parameter U_sp to represent the unloading time of each ship in each station 
U = [] # change U_cp into U_sp
for i in range(0,len(ship)):
    x = ship_data.iloc[i,1]
    U.append(U_cp.iloc[x-1,:].tolist())

## define auxiliary parameters for rolling planning horizon

In [4]:
timewindow = 10
fixed = 3 
overlap = timewindow - fixed
horizon = timewindow
num_iterations = math.ceil((len(S) - timewindow)/fixed) + 1 # calculate the number of iterations

## define decision variables

In [5]:
T = model.integer_var_matrix(S,P, name = "startTime") # arrival time of each ship on each station
D = model.integer_var_dict(S, name = "leaveTime") # each ship's leaving timetion
Z = model.binary_var_cube(S,S,P, name = "order2") # ship s is in order k and unloads in station p
O = model.binary_var_matrix(S,S, name = "order") # ship s is in order k

## Iteratively define the constraints, define the objective function, solve the model and shift the planning horizon

In [6]:
CPU_time = 0
for i in range(1,num_iterations+1): # Iteratively do 
    
    #------------------------------------------------------------------------------------
    # Define constraints
    #------------------------------------------------------------------------------------
    # order of each ship
    for s in S:
        #if s <= horizon:
            model.add_constraint(model.sum(Z[s,k,p] for k in S  for p in P) == 1)
            model.add_constraint(model.sum(O[s,k] for k in S) == 1)
    for k in S:
        if k <= horizon:
            model.add_constraint(model.sum(Z[s,k,p] for s in S  for p in P) == 1)
            model.add_constraint(model.sum(O[s,k] for s in S) == 1)
    
    for s in S:
        #if s <= horizon:
            for k in S:
                if k <= horizon:
                    model.add_constraint(model.sum(Z[s,k,p] for p in P) == O[s,k])
    
    # For ship in each position, it can enter the next station 
    # only when finished unloading (or just passing) at the current station 
    for p in range(1,len(P)):
        for k in S:
            if k <= horizon:
                model.add_constraint(T[k,p+1] >= T[k,p] + model.sum(Z[s,k,p]*U[s-1][p-1] for s in S) + 1 - model.sum(Z[s,k,p] for s in S))
    # The time for a ship to enter the open sea should 
    # not be less than the time to enter the last station and pass through this station.
    for k in S:
        if k <= horizon:
            model.add_constraint(T[k,len(P)] + model.sum(Z[s,k,len(P)]*U[s-1][len(P)-1] for s in S) + 1 - model.sum(Z[s,k,len(P)] for s in S) <= D[k])
    # For each station, the next ship comes only when the former ship left. 
    # Therefore, the entering time of the latter ship should be larger than the leaving time of the former ship.
    for p in range(1,len(P)):
        for k in range(1,len(S)):
            if k < horizon: 
                model.add_constraint(T[k+1,p] >= T[k,p+1])
    for k in range(1,len(S)):
        if k < horizon:
            model.add_constraint(T[k+1,len(P)] >= D[k])
    
    # T is non-negative
    for p in P:
        for k in S:
            if k <= horizon:
                model.add_constraint(T[k,p] >= 0)
    for p in P:
        for k in S:
            if k <= horizon:
                model.add_constraint(D[k] >= 0)
    # A ship can only enter the jetty when it arrived at the jetty. 
    # The entering time should be larger than the arrival time.
    for k in S:
        model.add_constraint(T[k,1] >= model.sum(O[s,k]*a[s-1] for s in S))
    #------------------------------------------------------------------------------------
    # Define the objective function 
    #------------------------------------------------------------------------------------
    horizon2 = min(horizon+1,len(S)+1)
    TotalTime = model.sum(D[k] - a[k-1] for k in range(1,horizon2))
    model.minimize(TotalTime)
    
    #------------------------------------------------------------------------------------
    # Solve the model
    #------------------------------------------------------------------------------------
    
    #model.set_time_limit(5) # set a time limit
    #model.parameters.mip.tolerances.mipgap.set(0.05) # set a larger optimality gap
    model.solve()

    #------------------------------------------------------------------------------------
    # Print the results and additional information
    #------------------------------------------------------------------------------------
    
    print("The solve status is", model.get_solve_status()) # was the model solved to optimality?
    print("The objective value is", model.objective_value) # print the objective function value
    print("The time limit was fixed to", model.get_time_limit()) # print the chosen time limit
    print("The optimality gap was fixed to", model.parameters.mip.tolerances.mipgap.get()) # print the chosen optimality gap
    print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time of the current iteration
    CPU_time = model.solve_details.time + CPU_time
    
    # order
    col = ["ship number","ship name","order"]
    res_Order = pd.DataFrame(columns = col)
    for s in S:
        res_Order.loc[s-1,"ship number"] = s
        res_Order.loc[s-1,"ship name"] = ship_data.iloc[s-1,0]    
        res_Order.loc[s-1,"order"] = round(sum(k*O[s,k].solution_value for k in range(1,horizon2))) 
    print(res_Order)
    
    #------------------------------------------------------------------------------------
    # Fix the decision variables of the current planning horizon
    #------------------------------------------------------------------------------------
    for s in S:
        # if s <= (horizon - overlap):
            for k in S:
                if k <= (horizon - overlap):
                    for p in P:
                        # for all periods until (horizon - overlap)
                        model.add_constraint(T[k,p] == T[k,p].solution_value)
                        model.add_constraint(D[k] == D[k].solution_value)
                        model.add_constraint(Z[s,k,p] == Z[s,k,p].solution_value)
                        model.add_constraint(O[s,k] == O[s,k].solution_value)
    
    #------------------------------------------------------------------------------------
    # Shift the planning horizon by (timewindow - overlap) periods
    #------------------------------------------------------------------------------------
    
    horizon = horizon + timewindow - overlap

print("The total CPU time is",CPU_time,"seconds.")

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 40176.000000 after 0.00 sec. (0.18 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 93 rows and 1085 columns.
MIP Presolve modified 242 coefficients.
Aggregator did 80 substitutions.
Reduced MIP has 263 rows, 932 columns, and 3691 nonzeros.
Reduced MIP has 894 binaries, 38 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (8.33 ticks)
Probing time = 0.00 sec. (2.71 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 263 rows, 932 columns, and 3691 nonzeros.
Reduced MIP has 894 binaries, 38 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (2.77 ticks)
Probing time = 0.00 sec. (2.69 ticks)
Clique table members: 1708.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 6 threads.
Root relaxation solution time = 0.02 sec. (10.84 ticks)


      0     0      150.5998   112    40229.0000       Cuts: 3      644   99.63%
      0     0      150.6074   103    40229.0000       Cuts: 2      654   99.63%
      0     0      150.6219    92    40229.0000       Cuts: 3      662   99.63%
*     0+    0                          161.0000      150.6219             6.45%
*     0+    0                          156.0000      150.6219             3.45%
      0     0        cutoff            156.0000      156.0000      662    0.00%
Elapsed time = 0.14 sec. (85.92 ticks, tree = 0.01 MB, solutions = 3)

Mixed integer rounding cuts applied:  3
Zero-half cuts applied:  6
Gomory fractional cuts applied:  1

Root node processing (before b&c):
  Real time             =    0.14 sec. (86.02 ticks)
Parallel b&c, 6 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.14 sec. (86.02 ticks)
The solve s

                          ------------
Total (root+branch&cut) =    0.22 sec. (108.00 ticks)
The solve status is JobSolveStatus.OPTIMAL_SOLUTION
The objective value is 254.0
The time limit was fixed to 1e+75
The optimality gap was fixed to 0.0001
The CPU time is 0.21899999998277053 seconds
   ship number  ship name order
0            1      Exxon     2
1            2       Nina    16
2            3      Kabul     1
3            4    Opladen     5
4            5    Tornado     3
5            6       Fony    19
6            7      Moers     4
7            8  Clausthal    14
8            9      Herta     6
9           10       Colt     7
10          11    Natalie    18
11          12    Duandra     8
12          13     Neptun    17
13          14    Onassis     9
14          15   Atlantic    11
15          16     Carina    10
16          17    Waldhof    12
17          18     Panega    20
18          19     Megara    13
19          20      Hansa    15
The total CPU time is 1.0160000000032